<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook demonstrates the concept of indirect bias detection and automated bias mitigation using IBM Watson OpenScale's auto-debias endpoint.

We make use of an HR hiring dataset, where the model will be unfairly biased against females and minorities. These two features are not used in the training of the model, and do not affect the prediction. However, their values are submitted as metadata with the prediction request, and are logged in the OpenScale datamart.

Using the training data, OpenScale detects correlations between the protected features and the model training features. OpenScale can then monitor the protected features for unfair bias, and optionally attempt to remove that bias by perturbing the correlated features if the prediction is submitted to the OpenScale-generated debiased endpoint.

The model will be configured as a pre-production model in OpenScale.

This notebook should be run in a Watson Studio project, using a Python 3.7 or above runtime environment. If you are viewing this in Watson Studio and do not see Python 3.7 or above in the upper right corner of your screen, please update the runtime now. It requires the following services:

 - IBM Watson OpenScale, configured to use a database on the cluster
 - Watson Machine Learning

# Setup <a name="setup"></a>

## Create a deployment space

**It is HIGHLY recommended that you create a new space for this project and model, because this script will remove existing spaces as machine learning service providers for OpenScale, which may interfere with other models you are monitoring if they are in the same space.**

All deployed models require a deployment space. Go to the **Deployments** section of your cluster to create a new space, or choose an existing one.

Click on the name of the space, then go to the **Settings tab**. Locate the **Space ID** and then click the icon to copy the ID to your clipboard. Paste your space ID between the quotation marks below.

In [ ]:
WML_SPACE_ID = '___PASTE_HERE___'

## Enter login credentials

Paste your credentials for the cluster below. They will be used to log into services for deploying the model and configuring OpenScale.

In [ ]:
WOS_CREDENTIALS = {
    "url": "___CLUSTER_URL___",
    "username": "___USERNAME___",
    "password": "___PASSWORD___",
    "version": "3.5"
}

## Package installation

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install --upgrade pyspark==2.4 --no-cache | tail -n 1

!pip install --upgrade pandas==0.25.3 --no-cache | tail -n 1
!pip install --upgrade requests==2.23 --no-cache | tail -n 1
!pip install numpy==1.16.4 --no-cache | tail -n 1
!pip install SciPy --no-cache | tail -n 1
!pip install lime --no-cache | tail -n 1
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

## Run the notebook

At this point, you can run the notebook step-by-step until you reach the instructions telling you to upload test data to OpenScale. If you encounter errors, restarting the kernel here and re-running may resolve them.

In [ ]:
import os
import base64
import json
import requests
from requests.auth import HTTPBasicAuth

In [ ]:
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

## Training data in Cloud Object Storage

### Cloud object storage details¶

The next cells contain the location of the model training data in cloud object storage. OpenScale needs access to the data for calculations on protected value correlations with model features, drift data, explainability, and more.

In [ ]:
IAM_URL = "https://iam.ng.bluemix.net/oidc/token"

In [ ]:
# masked
COS_API_KEY_ID = "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::"
COS_ENDPOINT = "https://s3.us.cloud-object-storage.appdomain.cloud"
BUCKET_NAME = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7"
FILE_NAME = "hr_training_data.csv"

# Load and explore data

In [ ]:
!rm hr_training_data.csv
!wget https://raw.githubusercontent.com/ericmartens/indirect-bias/main/data/hr_training_data.csv

## Explore data

In [ ]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="hr_training_data.csv", sep=",", header=True, inferSchema=True) 
df_data.head()

In [ ]:
print("Number of records: " + str(df_data.count()))

# Create a model

In [ ]:
spark_df = df_data
protected_attributes = ["Ethnicity", "Gender"]
for attr in protected_attributes:
    spark_df = spark_df.drop(attr)
columns = spark_df.columns
model_name = "Hiring Helper"
deployment_name = "Hiring Helper Deployment"

spark_df.printSchema()

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml import Pipeline, Model

cat_features = ['BusinessTravel', 'Department', 'Education', 'EducationField', 'RelevantEducationLevel', 'JobRole', 'JobLevel',\
                'MaritalStatus', 'OverTime', 'RequestedBenefits', 'PreferredSkills', 'JobType', 'SalaryExpectation',\
                'InterviewScore', 'ResumeScore'] 
num_features = ['Age', 'DistanceFromHome', 'NumCompaniesWorked', 'TotalWorkingYears', 'YearsAtCurrentCompany', 'RelevantExperience']
stages=[]

for feature in cat_features:
    string_indexer = StringIndexer(inputCol = feature, outputCol = feature + '_IX').setHandleInvalid("keep")
    encoder = OneHotEncoderEstimator(inputCols=[string_indexer.getOutputCol()], outputCols=[feature + "classVec"])
    stages += [string_indexer, encoder]

si_Label = StringIndexer(inputCol="HIRED", outputCol="encoded_label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)
stages.append(si_Label)

In [ ]:
assembler_inputs = [c + "classVec" for c in cat_features] + num_features
va_features = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages.append(va_features)

In [ ]:
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

In [ ]:
train_data.columns

In [ ]:
from pyspark.ml.classification import GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
classifier = RandomForestClassifier(labelCol="encoded_label", featuresCol="features")
stages.append(classifier)
stages.append(label_converter)
pipeline = Pipeline(stages=stages)
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
predictions.printSchema()
predictions.head()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluatorDT = BinaryClassificationEvaluator(labelCol="encoded_label", rawPredictionCol="rawPrediction")
accuracy = evaluatorDT.evaluate(predictions)

print("Accuracy = %g" % accuracy)

# Save and deploy the model

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.spaces.list(limit=10)

## Set the default space specified earlier in the notebook

In [ ]:
wml_client.set.default_space(WML_SPACE_ID)

In [ ]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == deployment_name:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
training_data_references = [
                {
                    "id": "product line",
                    "type": "s3",
                    "connection": {
                        "access_key_id": COS_API_KEY_ID,
                        "endpoint_url": COS_ENDPOINT,
                        "resource_instance_id":COS_RESOURCE_CRN
                    },
                    "location": {
                        "bucket": BUCKET_NAME,
                        "path": FILE_NAME,
                    }
                }
            ]

In [ ]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_2.4")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(model_name),
        wml_client._models.ConfigurationMetaNames.SPACE_UID: WML_SPACE_ID,
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_2.4",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "HIRED",
    }

In [ ]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_data=train_data, 
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

In [ ]:
published_model_details

## Create a model deployment

In [ ]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(deployment_name),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# Construct the scoring payload

In [ ]:
import pandas as pd

df = pd.read_csv("hr_training_data.csv")
df.head()

## Remove the sensitive attributes

In [ ]:
cols_to_remove = ['HIRED']
cols_to_remove.extend(protected_attributes)
cols_to_remove

## Create the meta data frame capturing the sensitive data

In [ ]:
meta_df = df[protected_attributes].copy()
meta_fields = meta_df.columns.tolist()
meta_values = meta_df[meta_fields].values.tolist()

## Construct the scoring payload comprising the meta fields

In [ ]:
def get_scoring_payload(no_of_records_to_score = 1):
    meta_payload = {
        "fields": meta_fields,
        "values": meta_values[:no_of_records_to_score]
    }

    for col in cols_to_remove:
        if col in df.columns:
            del df[col] 

    fields = df.columns.tolist()
    values = df[fields].values.tolist()

    payload_scoring = {"input_data": [{"fields": fields, "values": values[:no_of_records_to_score],"meta": meta_payload}]}  
    return payload_scoring

In [ ]:
deployment_uid

## Method to perform scoring

In [ ]:
def sample_scoring(no_of_records_to_score = 1):
    records_list=[]
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)
    print('Scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])
    print(json.dumps(scoring_response, indent=None))
    return payload_scoring, scoring_response

In [ ]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
def payload_logging(no_of_records_to_score = 1):
    records_list=[]
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    
    
    scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)
    time.sleep(10)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))
    if pl_records_count == 0:
        print("Payload logging did not happen, performing explicit payload logging.")
    
        #manual PL logging if automated logging does not work
        score_input=payload_scoring['input_data'][0]
        score_response=scoring_response['predictions'][0]
        pl_record = PayloadRecord(request=score_input, response=score_response, response_time=int(460))
        records_list.append(pl_record)
        wos_client.data_sets.store_records(data_set_id = payload_data_set_id, request_body=records_list)
        
        
        time.sleep(10)
        pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
        print("Number of records in the payload logging table: {}".format(pl_records_count))

## Score the model and print the scoring response

In [ ]:
sample_scoring(no_of_records_to_score = 1)

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

## Get a instance of the OpenScale SDK client

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],service_instance_id="00000000-0000-0000-0000-1611161485480737", authenticator=authenticator)
wos_client.version

In [ ]:
wos_client.data_marts.show()

In [ ]:
DB_CREDENTIALS = None

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.DB2,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DATABASE_CREDENTIALS['hostname'],
                        username=DATABASE_CREDENTIALS['username'],
                        password=DATABASE_CREDENTIALS['password'],
                        db=DATABASE_CREDENTIALS['database'],
                        port=DATABASE_CREDENTIALS['port'],
                        ssl=DATABASE_CREDENTIALS['ssl'],
                        sslmode=DATABASE_CREDENTIALS['sslmode'],
                        certificate_base64=DATABASE_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

In [ ]:
data_mart_details = wos_client.data_marts.list().result.data_marts[0]
data_mart_details.to_dict()

In [ ]:
wos_client.service_providers.show()

## Remove existing service provider connected with used WML instance.

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [ ]:
SERVICE_PROVIDER_NAME = "WML - Indirect Bias Testing"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook to showcase Indirect Bias functionality."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.
Note: You can bind more than one engine instance if needed by calling wos_client.service_providers.add method. Next, you can refer to particular service provider using service_provider_id.

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "pre_production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
print(wos_client.service_providers.get(service_provider_id).result)

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_id=deployment_uid, deployment_space_id = WML_SPACE_ID).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=WML_SPACE_ID)

## Subscriptions

Remove existing model subscriptions

This code removes previous subscriptions to the model to refresh the monitors with the new model and new data.

In [ ]:
wos_client.subscriptions.show()

## Remove the existing subscription

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [ ]:
feature_columns = cat_features + num_features
feature_columns

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="HIRED",
            probability_fields=["probability"],
            prediction_field="predictedLabel",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = FILE_NAME),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL})),
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
wos_client.data_sets.show()

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

# Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model.

In [ ]:
payload_logging(no_of_records_to_score = 10)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

## Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:
    
Which model feature to monitor One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%) Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Create Fairness Monitor Instance

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "features": [
        {
            "feature": "Gender",
            "majority": ["Male"],
            "minority": ["Female"]
        },
        {
            "feature": "Ethnicity",
            "majority": ["non-minority"],
            "minority": ["minority"]
        }
    ],
    "favourable_class": ["YES"],
    "unfavourable_class": ["NO"],
    "min_records": 100
}

thresholds = [
    {
        "metric_id": "fairness_value",
        "specific_values": [
           {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "Gender",
                        "key": "feature"
                    }
                ],
                "value": 80
            },
            {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "Ethnicity",
                        "key": "feature"
                    }
                ],
                "value": 80
            }
        ],
        "type": "lower_limit",
        "value": 80
    }
]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id

### Get Fairness Monitor Instance

In [ ]:
wos_client.monitor_instances.show()

### Drift Configuration

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

## Enable quality monitoring
The code below turns on the quality (accuracy) monitor and sets an alert threshold of 80%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [ ]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

## Configure Explainability
Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

# STOP HERE! Upload and evaluate test data

At this point, you can navigate to the **Instances** section of your cluster and open OpenScale. From the OpenScale Insights Dashboard, and select the **Hiring Deployment - Indirect Debias** model. From the **Actions** menu, choose **Evaluate now**. Select **from CSV** from the **Import** dropdown, and upload the [payload_100.csv](https://raw.githubusercontent.com/ericmartens/indirect-bias/main/data/payload_100.csv) file. Then click **Upload and evaluate**. The monitors will take a few minutes to run, but when the screen refreshes, you will see information on the test results for fairness, quality and drift, along with two generated explanations.

When the tests have finished running, you may continue with the steps below to test the auto-debiased endpoint.

## Test the model and auto-debiased endpoing results using a record that will produce a biased result

Now that we've run the model test, OpenScale's auto-debiased endpoint is active. We can see how this works by sending records to the deployed model that will produce a biased result, then sending the same records to the debiased endpoint and seeing that we get a different prediction.

First, we'll get the scoring endpoint for the deployed model.

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions

space_id = None
deployment_id = None
datamart_id = None
subscription_id = None
for subscription in subscriptions:
    if subscription.entity.deployment.name == deployment_name:
        deployment_id = subscription.entity.deployment.deployment_id
        space_id = subscription.entity.asset.url.split('?space_id=')[1].split('&version')[0]
        datamart_id = subscription.entity.data_mart_id
        subscription_id = subscription.metadata.id
        print("Deployment ID:", deployment_id)
        print("Space ID:", space_id)
        print("Datamart ID:", datamart_id)
        print("Subscription ID:", subscription_id)

In [ ]:
wml_client.set.default_space(space_id)

In [ ]:
fields = ['Age', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
          'EducationField', 'RelevantEducationLevel', 'JobLevel', 'JobRole', 'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'InterviewScore',
          'ResumeScore', 'RequestedBenefits', 'TotalWorkingYears', 'PreferredSkills', 'YearsAtCurrentCompany', 'RelevantExperience', 'JobType', 'SalaryExpectation']

Next, we'll get 50 records to send to our production model and our debiased endpoint.

In [ ]:
no_of_records_to_score = 50
scoring_payload = get_scoring_payload(no_of_records_to_score)

Send the records to the production model for scoring.

In [ ]:
scoring_response = wml_client.deployments.score(deployment_id, scoring_payload)

Next, we'll get an authentication token so we can use the auto-debiased endpoint.

In [ ]:
import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

token_url = WOS_CREDENTIALS['url'] + '/v1/preauth/validateAuth'
headers = {}
headers["Accept"] = "application/json"
auth = HTTPBasicAuth(WOS_CREDENTIALS['username'], WOS_CREDENTIALS['password'])
response = requests.get(token_url, headers=headers, auth=auth, verify=False)
json_data = response.json()
access_token = json_data['accessToken']

In [ ]:
DEBIASING_PREDICTIONS_URL = WOS_CREDENTIALS['url'] + "/openscale/{0}/v2/subscriptions/{1}/predictions".format(data_mart_id,subscription_id)
print(DEBIASING_PREDICTIONS_URL)

headers = {}
headers["Content-Type"] = "application/json"
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer {}".format(access_token)

debiased_scoring_payload = scoring_payload['input_data'][0]
response = requests.post(DEBIASING_PREDICTIONS_URL, data=json.dumps(debiased_scoring_payload), headers=headers, verify=False)

Find the and print the records where the model prediction differs from the debiased prediction.

In [ ]:
predictedLabel_index = response.json()['fields'].index('predictedLabel')
debiased_prediction_index = response.json()['fields'].index('debiased_prediction')

for j in range(no_of_records_to_score):
    scored_record = response.json()['values'][j]
    predictedLabel = scored_record[predictedLabel_index]
    debiased_prediction = scored_record[debiased_prediction_index]
    if predictedLabel != debiased_prediction:
        print('==========')
        print(scored_record)
        print('predictedLabel:' + str(predictedLabel) + ', debiased_prediction=' + str(debiased_prediction))
        print('==========')